In [111]:
import json
import requests
import pandas as pd
import random

In [2]:
url = "https://datahub.io/core/country-list/r/data.json"

In [3]:
countries = requests.get(url)
countries.status_code

200

In [4]:
list_ = countries.json()
countries_list = [d['Name'] for d in list_]

In [ ]:
country = random.choice(countries_list)
country

In [128]:
country = "Hungary"

In [129]:
api_url = f"https://inspirehep.net/api/institutions?q={country}"

In [130]:
inst_link = requests.get(api_url)

In [131]:
legacy_ICN = [
    inst.get('metadata').get('legacy_ICN') for inst in inst_link.json().get('hits').get('hits')
    if inst.get('metadata').get('number_of_papers') > 0
]

In [132]:
first_inst = legacy_ICN[0]
# Replacing the spaces " " with "+"
first_inst = first_inst.replace(" ", '+')

In [133]:
articles_url = f"https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+{first_inst}+and+ac+1->+10"

In [134]:
articles_url

'https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+Baja+Observ.,+Hungary+and+ac+1->+10'

In [135]:
articles = requests.get(articles_url)

In [136]:
# The first one of the articles json
art1 = articles.json().get('hits').get('hits')[0].get('metadata')

In [137]:
links_authors = [author.get('record').get('$ref') for author in art1.get('authors')]

In [138]:
links_authors

['https://inspirehep.net/api/authors/2234871',
 'https://inspirehep.net/api/authors/2089437',
 'https://inspirehep.net/api/authors/2216197',
 'https://inspirehep.net/api/authors/2337176',
 'https://inspirehep.net/api/authors/2268586',
 'https://inspirehep.net/api/authors/2337179']

In [159]:
def filter_name(info):
  try:
    return info.get('name').get('value') 
  except:
    return None

def filter_email(info):
  try:
    return info.get('email_addresses')[0].get('value')
  except:
    return None

def filter_position(info):
  try:
    return info.get('positions')[0].get('rank')
  except:
    return None

def filter_institution(info):
  try:
    return info.get('positions')[0].get('institution')
  except:
    return None

def filter_start(info):
  try:
    return info.get('positions')[0].get('start_date')
  except:
    return None

def filter_end(info):
  try:
    return info.get('positions')[0].get('end_date')
  except:
    return None


In [163]:
def make_df(links_authors):
  json_authors = [requests.get(aut).json().get('metadata') for aut in links_authors]

  names = [filter_name(info) for info in json_authors]
  emails = [filter_email(info)  for info in json_authors]
  actual_positions = [filter_position(info)  for info in json_authors]
  institutions = [filter_institution(info)  for info in json_authors]
  start_dates = [filter_start(info) for info in json_authors]
  end_dates = [filter_end(info)  for info in json_authors]

  dicc = {
      'Name': names,
      'Email': emails,
      'Actual_position': actual_positions,
      'Institution': institutions,
      'Start_date': start_dates,
      'End_date': end_dates
  }

  df = pd.DataFrame(dicc)

  return df


In [164]:
df = make_df(links_authors)

In [165]:
df

,Name,Email,Actual_position,Institution,Start_date,End_date
0,"Csizmadia, Sz.",None,None,None,None,None
1,"Nagy, Zs.",None,None,None,None,None
2,"Borkovits, T.",None,None,None,None,None
3,"Hegedus, T.",None,None,None,None,None
4,"Biro, I.B.",None,None,None,None,None
5,"Kiss, Z.T.",None,None,None,None,None


## Second country

In [167]:
country = random.choice(countries_list)
country

'Switzerland'

In [168]:
country = "Switzerland"
api_url = f"https://inspirehep.net/api/institutions?q={country}"
inst_link = requests.get(api_url)
legacy_ICN = [
    inst.get('metadata').get('legacy_ICN') for inst in inst_link.json().get('hits').get('hits')
    if inst.get('metadata').get('number_of_papers') > 0
]
first_inst = legacy_ICN[0]
# Replacing the spaces " " with "+"
first_inst = first_inst.replace(" ", '+')
articles_url = f"https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+{first_inst}+and+ac+1->+10"
articles = requests.get(articles_url)
# The first one of the articles json
art1 = articles.json().get('hits').get('hits')[0].get('metadata')
links_authors = [author.get('record').get('$ref') for author in art1.get('authors')]

df2 = make_df(links_authors)

In [169]:
df2

,Name,Email,Actual_position,Institution,Start_date,End_date
0,"Cornu, Cynthia",None,None,None,None,None
1,"Deldicque, Christian",None,None,None,None,None
2,"Gladki, Maciej",None,None,None,None,None
3,"Holzner, Andre Georg",andre.georg.holzner@cern.ch,None,"UC, San Diego",2010,None
4,"Perrenoud, J.-L.",None,None,None,None,None
5,"Reis, Thomas",thomas.reis@stfc.ac.uk,JUNIOR,Rutherford,2018,None
6,"Sakulin, Hannes",Hannes.Sakulin@cern.ch,None,CERN,None,None
7,"Vougioukas, Michail",None,None,None,None,None
